In [ ]:
!rm -r /content/mouse_move_gan
!git clone https://764bf8496bb32ddc2d4153fd96508a5e971f7834@github.com/jchen42703/mouse_move_gan.git
%cd mouse_move_gan
!pip install .
%cd /content
!pip show pymousegan

rm: cannot remove '/content/mouse_move_gan': No such file or directory
Cloning into 'mouse_move_gan'...
remote: Enumerating objects: 581, done.
remote: Counting objects: 100% (581/581), done.
remote: Compressing objects: 100% (372/372), done.
remote: Total 581 (delta 343), reused 431 (delta 193), pack-reused 0
Receiving objects: 100% (581/581), 33.63 MiB | 17.05 MiB/s, done.
Resolving deltas: 100% (343/343), done.
/content/mouse_move_gan
Processing /content/mouse_move_gan
  Created wheel for pymousegan: filename=pymousegan-0.0.1-cp36-none-any.whl size=21828 sha256=4ecf6b9c4f323e4879cbaa751d42ef5800a14d97c258f54f56cb8523b8443ecf
  Stored in directory: /tmp/pip-ephem-wheel-cache-wz6h11_j/wheels/cf/a1/ad/f73bd409ec8c6c159e81d664f6837c86ae7a7f9c63aae218c5
Successfully built pymousegan
/content
Name: pymousegan
Version: 0.0.1
Summary: Library for models for generating random mouse movements
Home-page: UNKNOWN
Author: Joseph Chen
Author-email: jchen42703@gmail.com
License: MIT
Location: /usr

In [ ]:
!nvidia-smi

Wed Dec 23 07:12:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!rm -r model_out/

In [ ]:
# data
import numpy as np
coords_dt = np.load('/content/mouse_move_gan/data/coords_dt.npy')
coords_dt.shape, coords_dt.min(), coords_dt.max()

((10963, 100, 3), 0.0, 1.0)

## Download weights

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
# file_id = '1M58-nFuFG4g-KKKpkd-iIYxg5gn4aNfd'
# gdd.download_file_from_google_drive(file_id=file_id,
#                                     dest_path='./model_105001_125001.zip',
#                                     unzip=True)
# file_id = '1Do9EeBK_jZ1YIs_acO2YCSlSfMcbNjN5'
# gdd.download_file_from_google_drive(file_id=file_id,
#                                     dest_path='./model_165001_185001.zip',
#                                     unzip=True)

# file_id = '1kQtjxP_WQrjx8WbL_cF4dX0DY8T7ZTRc'
# gdd.download_file_from_google_drive(file_id=file_id,
#                                     dest_path='./model_95001_105001.zip',
#                                     unzip=True)

Unzipping...Done.


In [ ]:
model_paths = {
    'discrim_path': '/content/model_out/discrim_105001_weights.h5',
    'gen_path': '/content/model_out/gen_105001_weights.h5',
    'combined_path': '/content/model_out/combined_105001_weights.h5'
}

# GAN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from pymousegan.models.gans import AdditiveBasicGAN
from pymousegan.models.lstm import BidirectionalLSTMDecoderGenerator, \
  LSTMDecoderGenerator, BidirectionalLSTMDiscriminator
from pymousegan.io.data_aug import scale_translate_v2, scale_translate_v3

seed = 420
np.random.seed(seed)
tf.random.set_seed(seed)

seq_shape = (100, 3)
noise_size = (100, 100)
d_opt = Adam(lr=1e-4, beta_1=0.5)
g_opt = Adam(lr=1e-5, beta_1=0.5)

def unique_batch_reflect(real_seqs: np.array):
    """Reflects but makes each path in the sequence have unique reflection
    factor. Slower than reflect but may return more diverse paths.
    """
    reflected = np.zeros(real_seqs.shape)
    for i, path in enumerate(real_seqs):
        reflectX = np.random.choice([-1, 1])
        reflectY = np.random.choice([-1, 1])
        reflected[i] = path * np.array([reflectX, reflectY, 1])
    return reflected

# model_paths = {
#     'discrim_path': '/content/model_out/discrim_105001_weights.h5',
#     'gen_path': '/content/model_out/gen_105001_weights.h5',
#     'combined_path': '/content/model_out/combined_105001_weights.h5'
# }
model_paths = {}

discrim = BidirectionalLSTMDiscriminator(seq_shape, hidden_units_list=[256, 128],
                                         minibatch_discrim_units=10, minibatch_discrim_row_size=3)
generator = BidirectionalLSTMDecoderGenerator(noise_size, seq_shape,
                                              hidden_units_list=[256, 128],
                                              output_act='tanh')

gan = AdditiveBasicGAN(discrim, generator, d_opt, g_opt, data_aug=unique_batch_reflect,
                       model_paths=model_paths)

gan.discriminator.model.summary(), gan.generator.model.summary()

Creating a discrminator with 2 LSTM layers.
Creating a generator with 2 LSTM layers.
Model: "bidirectional_lstm_discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 100, 3)]          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 100, 512)          532480    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 256)               656384    
_________________________________________________________________
minibatch_discrimination_1 ( (None, 266)               7680      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 267       
Total params: 1,196,811
Trainable params: 0
Non-trainable params: 1,196,811
_________________________________________________________________
Model

(None, None)

In [ ]:
!rm -r model_out/

In [ ]:
!mkdir /content/model_out/
gan.train(coords_dt, num_epochs=15001, batch_size=16, sample_interval=100, output_dir='/content/model_out', save_format='h5')

In [ ]:
!mkdir /content/model_out/
gan.train(coords_dt, num_epochs=35001, batch_size=16, sample_interval=100, output_dir='/content/model_out', save_format='h5', initial_epoch=15001)

In [ ]:
!mkdir /content/model_out/
gan.train(coords_dt, num_epochs=55001, batch_size=16, sample_interval=100, output_dir='/content/model_out', save_format='h5', initial_epoch=35001)

In [ ]:
!mkdir /content/model_out/
gan.train(coords_dt, num_epochs=75001, batch_size=16, sample_interval=100, output_dir='/content/model_out', save_format='h5', initial_epoch=55001)

In [ ]:
!mkdir /content/model_out/
gan.train(coords_dt, num_epochs=95001, batch_size=16, sample_interval=100, output_dir='/content/model_out', save_format='h5', initial_epoch=75001)

In [ ]:
!mkdir /content/model_out/
gan.train(coords_dt, num_epochs=125001, batch_size=16, sample_interval=100, output_dir='/content/model_out', save_format='h5', initial_epoch=95001)

In [ ]:
!mkdir /content/model_out/
gan.train(coords_dt, num_epochs=155001, batch_size=16, sample_interval=100, output_dir='/content/model_out', save_format='h5', initial_epoch=125001)

In [ ]:
!mkdir /content/model_out/
gan.train(coords_dt, num_epochs=185001, batch_size=16, sample_interval=100, output_dir='/content/model_out', save_format='h5', initial_epoch=155001)

In [ ]:
!zip -r model_0_15001.zip model_out/
# !zip -r model_15001_35001.zip model_out/
# !zip -r model_35001_55001.zip model_out/
# !zip -r model_55001_75001.zip model_out/
# !zip -r model_75001_95001.zip model_out/
# !zip -r model_125001_155001.zip model_out/
# !zip -r model_155001_185001.zip model_out/

# !zip -r model_0_5001.zip model_out/
# !zip -r model_15001_25001.zip model_out/
# !zip -r model_25001_35001.zip model_out/
# !zip -r model_35001_45001.zip model_out/
# !zip -r model_45001_55001.zip model_out/
# !zip -r model_55001_65001.zip model_out/
# !zip -r model_65001_75001.zip model_out/
# !zip -r model_75001_85001.zip model_out/
# !zip -r model_95001_105001.zip model_out/
# !zip -r model_105001_125001.zip model_out/
# !zip -r model_125001_145001.zip model_out/
# !zip -r model_145001_165001.zip model_out/
# !zip -r model_165001_185001.zip model_out/
# !zip -r model_185001_205001.zip model_out/
# !zip -r model_205001_235001.zip model_out/
# !zip -r model_235001_265001.zip model_out/


In [ ]:
#### AUTHENTICATION ###
# saving weights and uploading them to gdrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
folders = {
    'Weights': '18KXnIoGhXYSb0l6qkjQmvphDpkiJ8yaa',
    'BasicGAN': '1jz_AOYPyg3Oco6KpnS_MOKQUzg3ZcUXB',
    'AdditiveBasicGAN': '1WfdGeWb39l_SfkZE9vV23zuZ7jAQaqRv'
}

def upload_file(fname, folder_id=folders['Weights']):
  file_dict = {'title': fname, 'parents': [{'kind': 'drive#fileLink', 'id': folder_id}]}
  f = drive.CreateFile(file_dict)
  f.SetContentFile(fname)
  f.Upload()
  print('Uploaded: {0}'.format(fname))


In [ ]:
fname = 'model_0_15001.zip'
# fname = 'model_15001_35001.zip'
# fname = 'model_35001_55001.zip'
# fname = 'model_55001_75001.zip'
# fname = 'model_75001_95001.zip'
# fname = 'model_95001_125001.zip'
# fname = 'model_125001_155001.zip'
# fname = 'model_155001_185001.zip'

# fname = 'model_0_5001.zip'
# fname = 'model_5001_15001.zip'
# fname = 'model_15001_25001.zip'
# fname = 'model_25001_35001.zip'
# fname = 'model_35001_45001.zip'
# fname = 'model_45001_55001.zip'
# fname = 'model_55001_65001.zip'
# fname = 'model_65001_75001.zip'
# fname = 'model_75001_85001.zip'
# fname = 'model_75001_95001.zip'
# fname = 'model_125001_145001.zip'
# fname = 'model_145001_165001.zip'
# fname = 'model_165001_185001.zip'
# fname = 'model_185001_205001.zip'
# fname = 'model_205001_235001.zip'
# fname = 'model_235001_265001.zip'

upload_file(fname, folders['AdditiveBasicGAN'])

Uploaded: model_0_15001.zip
